In [83]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import patsy

In [123]:
# Idea is to merge these 9 treaty datasets, as modified for a date difference, and 4 independent variable datasets.
iccpr = pd.read_excel("../data/UnderlyingData_ICCPR_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
icescr = pd.read_excel("../data/UnderlyingData_ICESCR_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
icerd = pd.read_excel("../data/UnderlyingData_ICERD_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
cedaw = pd.read_excel("../data/UnderlyingData_CEDAW_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
crc = pd.read_excel("../data/UnderlyingData_CRC_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
catc = pd.read_excel("../data/UnderlyingData_CAT_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
crpd = pd.read_excel("../data/UnderlyingData_CRPD_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
crmw = pd.read_excel("../data/UnderlyingData_ICRMW_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)
ced = pd.read_excel("../data/UnderlyingData_CPED_OHCHR_19_09_2020.xls",skiprows=[0],nrows=198)

In [124]:
#Critical dates as gathered in a quick search. There is probably a mistake here, but it is just a matter
# of fact checkind latter
iccpr['sdate'] = pd.to_datetime("1966-12-06")
icescr['sdate'] =  pd.to_datetime("1966-12-06")
icerd['sdate'] =  pd.to_datetime("1965-12-21")
cedaw['sdate'] =  pd.to_datetime("1980-03-01")
crc['sdate'] =  pd.to_datetime("1989-11-20")
catc['sdate'] =  pd.to_datetime("1984-12-10")
crpd['sdate'] =  pd.to_datetime("1990-12-18")
crmw['sdate'] =  pd.to_datetime("2007-02-06")
ced['sdate'] =  pd.to_datetime("2007-03-30")

In [125]:
dataframes = [iccpr, icescr, icerd, cedaw, crc, catc, crpd, crmw, ced]
dataframes_names = 'iccpr, icescr, icerd, cedaw, crc, catc, crpd, crmw, ced'.split(', ')

In [126]:
#Getting the time differences
for df in dataframes:
    df['difference'] = df.apply(lambda row: row['Date of Ratification/Accession'] - row['sdate'], axis=1)

In [127]:
#Leaving only time difference
for df in dataframes:
    try:
        df.drop(['Date of Signature (dd/mm/yyyy)', 'Date of Ratification/Accession', 'Date of acceptance of individual communications procedure', 'sdate'], axis=1, inplace=True)
        df.drop('Date of acceptance of inquiry procedure', axis=1, inplace=True)
    except:
        pass

In [132]:
# I wanted to transform "Country" to an index of rows to facilitate merging
for df in dataframes:
    df.set_index(keys='Country', inplace=True)

In [133]:
full_df = pd.concat(dataframes, keys=dataframes_names, axis=1)

In [134]:
full_df

,iccpr,icescr,icerd,cedaw,crc,catc,crpd,crmw,ced
,difference,difference,difference,difference,difference,difference,difference,difference,difference
Country,,,,,,,,,
Afghanistan,5893 days,5893 days,6406 days,8404 days,1589 days,842 days,7945 days,NaT,NaT
Albania,9068 days,9068 days,10368 days,5184 days,829 days,3439 days,8091 days,119 days,223 days
Algeria,8316 days,8316 days,2246 days,5926 days,1243 days,1737 days,6926 days,-656 days,NaT
Andorra,14535 days,NaT,14885 days,6164 days,2234 days,7956 days,8484 days,NaT,NaT
Angola,9166 days,9166 days,19643 days,2391 days,380 days,12714 days,8553 days,NaT,NaT
...,...,...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),4173 days,4173 days,658 days,1157 days,297 days,2422 days,8316 days,3549 days,NaT
Viet Nam,5771 days,5771 days,6014 days,718 days,100 days,11014 days,8815 days,NaT,NaT


In [96]:
crc

,Country,difference
0,Afghanistan,1589 days
1,Albania,829 days
2,Algeria,1243 days
3,Andorra,2234 days
4,Angola,380 days
...,...,...
193,Venezuela (Bolivarian Republic of),297 days
194,Viet Nam,100 days
195,Yemen,527 days
196,Zambia,746 days


In [116]:
crpd

,difference
Country,
Afghanistan,7945 days
Albania,8091 days
Algeria,6926 days
Andorra,8484 days
Angola,8553 days
...,...
Venezuela (Bolivarian Republic of),8316 days
Viet Nam,8815 days
Yemen,6673 days
